<div align="center">
<img src="https://camo.githubusercontent.com/473dd9f992924d27457650251786464f72e54121ac6e9210add0f483ca849277/68747470733a2f2f692e696d6775722e636f6d2f3765523750616e2e706e67" width="40%">  
</div>

# Getting started with Petals

This notebook will guide you through the basics of Petals &mdash; a system for inference and fine-tuning 100B+ language models without the need to have high-end GPUs. With Petals, you can join compute resources with other people over the Internet and run large language models such as 176B-parameter [BLOOM](https://huggingface.co/bigscience/bloom) or [BLOOMZ](https://huggingface.co/bigscience/bloomz), which are of the same size as GPT-3.

💬 If you meet any issues while running this notebook, let us know in the **[#running-a-client](https://discord.gg/J29mCBNBvm)** channel of our Discord!

So, let's get started! First, let's install [the Petals package](https://github.com/bigscience-workshop/petals):


In [4]:
%pip install -q petals
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 1. The easiest way to generate text 🚀

Let's start with the easiest task &mdash; creating a __`DistributedBloom`__ model and using it for generating text.

This machine will download a small part of the model weights (~8 GB out of 352 GB) and rely on other computers in the network to run the rest of the model. Downloading the local part of the weights usually takes ~3 minutes.

🧑‍🏫 __Note:__ We suggest to start with the regular BLOOM, but you can also use [BLOOMZ](https://huggingface.co/bigscience/bloomz) &mdash; a version of BLOOM fine-tuned to better follow human instructions in the zero-shot regime. You would need to set `MODEL_NAME = "bigscience/bloomz-petals"` to load this model.

In [5]:
import torch
from transformers import BloomTokenizerFast 
from petals import DistributedBloomForCausalLM
import numpy as np

MODEL_NAME = "bigscience/bloom-petals"
tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME)
model = model.cuda()

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.19G [00:00<?, ?B/s]

Now, let's try to generate something by calling __`model.generate()`__ method.

The first call to this method takes ~5 sec to connect to the Petals swarm. Once we do that, you should expect generation speed of 1&ndash;1.5 sec/token. If you don't have enough GPUs to host the entire model, this is much faster than what you get with other methods, such as offloading (which takes at least 10&ndash;20 sec/token).

In [6]:
inputs = tokenizer('A cat in French is "', return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=3)
print(tokenizer.decode(outputs[0]))

A cat in French is "chat",


The `model.generate()` method runs **greedy** generation by default. However, you can choose other generation methods like **top-p/top-k sampling** or **beam search** by passing the corresponding parameters (you'll see an example in a bit). You can even implement custom generation methods (we'll cover that in **Step 5**).

🔏 **Note:** Your data is processed by other people in the public swarm. Learn more about privacy [here](https://github.com/bigscience-workshop/petals/wiki/Security,-privacy,-and-AI-safety). For sensitive data, you can set up a [private swarm](https://github.com/bigscience-workshop/petals/wiki/Launch-your-own-swarm) among people you trust.

In [8]:
import jsonlines

question,passage,answer=[],[],[]
with jsonlines.open('train.jsonl') as f:
    for line in f.iter():
        question.append(line['question']) # or whatever else you'd like to do
        passage.append(line['passage'])
        answer.append(line['answer'])

In [12]:
prompt="passage: \"The tower's tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure's weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries.\"\nquestion: \"was the leaning tower of pisa built leaning\"\nanswer: false\n\npassage: \"In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901.\"\nquestion: \"does each australian state have its own constitution\"\nanswer: true\n\npassage: \"Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be official when the Senate receives a signed nomination letter from the president naming the nominee, which is then entered in the Senate's record. There have been 37 unsuccessful nominations to the Supreme Court of the United States. Of these, 11 nominees were rejected in Senate roll-call votes, 11 were withdrawn by the president, and 15 lapsed at the end of a session of Congress. Six of these unsuccessful nominees were subsequently nominated and confirmed to other seats on the Court. Additionally, although confirmed, seven nominees either declined office or (in one instance) died before assuming office.\"\nquestion: \"has any supreme court nominee not been confirmed\"\nanswer: true\n\npassage: \"This list contains the top 50 accounts with the most followers on the social photo-sharing platform Instagram. As of October 2018, the most followed user is Instagram's own account, with over 260 million followers. Cristiano Ronaldo is the most followed individual, with over 144 million followers. Twelve accounts have exceeded 100 million followers on the site.\"\nquestion: \"does anyone on instagram have 1 billion followers\"\nanswer: false\n\npassage: \"Creme eggs are available annually between 1 January and Easter Day. In the UK in the 1980s, Cadbury made Creme Eggs available year-round but sales dropped and they returned to seasonal availability.\"\nquestion: \"can you buy cadburys creme eggs all year round\"\nanswer: false\n\npassage: \"Station 19 is an American action-drama television series created by Stacy McKee for ABC. McKee, Shonda Rhimes, Betsy Beers, and Paris Barclay serve as executive producers on the series, which is the second spin-off to Grey's Anatomy. Set in Seattle, the series focuses on the lives of the men and women at Seattle Fire Station 19. The series is produced by Shondaland and ABC Studios, with McKee serving as showrunner.\"\nquestion: \"is station 19 a grey's anatomy spin off\"\nanswer: true"

In [16]:
pmt = "\n\npassage: " + passage[0] + "\nquestion: " + question[0] + "\nanswer: "
prompt + pmt

'passage: "The tower\'s tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure\'s weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries."\nquestion: "was the leaning tower of pisa built leaning"\nanswer: false\n\npassage: "In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901."\nquestion: "does each australian state have its own constitution"\nanswer: true\n\npassage: "Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be officia

In [18]:
inputs = tokenizer(prompt+pmt, return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=3)
test = tokenizer.decode(outputs[0])

In [37]:
print(test[-10:-6])

true


In [38]:
output = []
for i in range(30,60):
  pmt = "\n\npassage: " + passage[i] + "\nquestion: " + question[i] + "\nanswer: "
  inputs = tokenizer(prompt+pmt, return_tensors="pt")["input_ids"].cuda()
  response = model.generate(inputs, max_new_tokens=3)
  output.append(response)

In [78]:
print(tokenizer.decode(output[1][0]))

passage: "The tower's tilt began during construction in the 12th century, caused by an inadequate foundation on ground too soft on one side to properly support the structure's weight. The tilt increased in the decades before the structure was completed in the 14th century. It gradually increased until the structure was stabilized (and the tilt partially corrected) by efforts in the late 20th and early 21st centuries."
question: "was the leaning tower of pisa built leaning"
answer: false

passage: "In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901."
question: "does each australian state have its own constitution"
answer: true

passage: "Justices are nominated by the president and then confirmed by the U.S. Senate. A nomination to the Court is considered to be official when the 

In [75]:
predict = []
pred = [tokenizer.decode(i[0]) for i in output]
for i in pred:
  predict.append(i[-10:-6])

In [76]:
prediction = [predict[i] in ('true') for i in range(30)]

In [77]:
validation = np.array(answer[30:60]) == np.array(prediction)
validation.sum() / len(validation)

0.7333333333333333